In [1]:
from helpers import get_entries_fig, get_exits_fig
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import streamlit as st

In [2]:
df = pd.read_csv('../csv/trading.csv', index_col='date', parse_dates=True, infer_datetime_format=True)
df['signal'] = df['pred'].diff()
df.dropna(inplace=True)
df.head()

,close,test,pred,signal
date,,,,
2019-04-05,2892.739990,1,1,0.0
2019-04-08,2895.770020,0,1,0.0
2019-04-09,2878.199951,1,1,0.0
2019-04-10,2888.209961,1,1,0.0
2019-04-11,2888.320068,1,1,0.0


In [3]:
def plot_trades(data=pd.DataFrame, title='Trades View'):

    df = data.copy()

    entries = df[df['signal'] == 1.0]['close']
    entries.rename('Buy', inplace=True)

    entries_fig = get_entries_fig(entries)

    exits = df[df['signal'] == -1.0]['close']
    exits.rename('Sell', inplace=True)

    exits_fig = get_exits_fig(exits)

    df = df.drop(columns=['signal'])

    price_sma_fig = px.line(df)

    all_figs = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_figs.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title
    )

    return all_figs


In [4]:
def calculate_portfolio(data=pd.DataFrame, initial_capital=10000, share_size=100):

    df = data.copy()

    initial_capital = float(initial_capital)

    df['Position'] = share_size * df['signal']
    df['Entry/Exit Position'] = df['Position'].diff()
    df['Holdings'] = df['close'] * df['Position']
    df['Cash'] = (
        initial_capital - (df['close'] * df['Entry/Exit Position']).cumsum()
    )
    df['Portfolio Total'] = df['Cash'] + df['Holdings']
    df['Actual Returns'] = df['close'].pct_change()
    df['Actual Cumulative Returns'] = (
        1 + df['Actual Returns']
    ).cumprod() - 1
    df['Algorithm Returns'] = df['Actual Returns'] * df['signal']
    df['Algorithm Cumulative Returns'] = (
        1 + df['Algorithm Returns']
    ).cumprod() - 1

    df = df.dropna().sort_index(axis='columns')

    return df

In [5]:
xgb_ptf = calculate_portfolio(df)
plot_trades(df.drop(columns=['pred', 'test']))

In [6]:
def plot_portfolio(data=pd.DataFrame, title='Portfolio Performance'):

    df = data.copy()

    entries = df[df['signal'] == 1.0]['Portfolio Total']
    entries.rename('Buy', inplace=True)

    entries_fig = get_entries_fig(entries)

    exits = df[df['signal'] == -1.0]['Portfolio Total']
    exits.rename('Sell', inplace=True)

    exits_fig = get_exits_fig(exits)

    price_sma_fig = px.line(df[['Portfolio Total']])

    all_fig = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig

In [7]:
def plot_returns(data=pd.DataFrame, title='Portfolio Returns'):
    """
    Plots algorithmic cumulative returns and buy & hold cumulative returns
    Input data must be a df made by `calculate_portfolio()` 
    """

    df = data.copy()

    returns_fig = px.line(
        df[['Actual Cumulative Returns', 'Algorithm Cumulative Returns']]
    )

    all_fig = go.Figure(
        data=returns_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig

In [8]:
plot_returns(xgb_ptf)

In [9]:
plot_portfolio(xgb_ptf)

In [10]:
xgb_ptf = calculate_portfolio(df)
plot_trades(df.drop(columns=['pred', 'test']))

In [11]:
xgb_ptf

,Actual Cumulative Returns,Actual Returns,Algorithm Cumulative Returns,Algorithm Returns,Cash,Entry/Exit Position,Holdings,Portfolio Total,Position,close,pred,signal,test
date,,,,,,,,,,,,,
2019-04-08,0.001047,0.001047,0.000000,0.000000,10000.000000,0.0,0.000000,10000.000000,0.0,2895.770020,1,0.0,0
2019-04-09,-0.005026,-0.006067,0.000000,-0.000000,10000.000000,0.0,0.000000,10000.000000,0.0,2878.199951,1,0.0,1
2019-04-10,-0.001566,0.003478,0.000000,0.000000,10000.000000,0.0,0.000000,10000.000000,0.0,2888.209961,1,0.0,1
2019-04-11,-0.001528,0.000038,0.000000,0.000000,10000.000000,0.0,0.000000,10000.000000,0.0,2888.320068,1,0.0,1
2019-04-12,0.005071,0.006609,-0.006609,-0.006609,300740.991211,-100.0,-290740.991211,10000.000000,-100.0,2907.409912,0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,0.355480,-0.011543,-0.027512,-0.000000,97745.727539,100.0,0.000000,97745.727539,0.0,3921.050049,0,0.0,1
2022-07-27,0.390934,0.026156,-0.027512,0.000000,97745.727539,0.0,0.000000,97745.727539,0.0,4023.610107,0,0.0,1
2022-07-28,0.407811,0.012133,-0.027512,0.000000,97745.727539,0.0,0.000000,97745.727539,0.0,4072.429932,0,0.0,1
